In [1]:
import json
import os
import logging
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from time import time
from tqdm import tqdm

In [2]:
# Model: CNN
class CNN:
    def initiate(self, n_classes):
        model = Sequential([
              ## Convolutional Layers
              Conv2D(filters=32, kernel_size=(5,5), padding='same', input_shape=(28,28,1)),
              MaxPooling2D(pool_size=(2,2)),
              Conv2D(filters=64, kernel_size=(5,5), padding='same'), 
              MaxPooling2D(pool_size=(2,2)),

              ## Fully Connected Layer
              Flatten(),
              Dense(512, activation='relu'), 
              Dense(n_classes, activation='softmax')
            ])

        return model
    
# Model: 2NN
class NN2Layers:
    def initiate(self, n_classes):
        model = Sequential([
                            Flatten(input_shape=(28,28)), 
                            Dense(200, activation='relu'), 
                            Dense(200, activation='relu'), 
                            Dense(n_classes, activation='softmax')
        ])

        return model

In [3]:
# Generate data based on chosen setting
def generate_data(data, use_val, setting, n_clients):

    # Load data from Tensorflow Library
    if data == "MNIST":
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        
        x_train = x_train.astype("float32") / 255
        x_test = x_test.astype("float32") / 255
        
        y_train = y_train.astype("float32")
        y_test = y_test.astype("float32")
    
    # Expand dims to add batch axis
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    
    # Divide train into val
    if use_val == True:
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=len(x_test), random_state=42)
    else:
        x_val, y_val = None, None
    
    # Distribute data based on setting
    if setting == "IID":
        # Shuffle train data
        x_train_shuffled, y_train_shuffled = utils.shuffle(x_train, y_train, random_state=21)
        
        # Partitioned proxy data to clients, each receiving 600 examples
        clients_data, clients_label = {}, {}
        list_data = np.array_split(x_train_shuffled, n_clients)
        list_label = np.array_split(y_train_shuffled, n_clients)

        # Distribute the data to all clients
        for i in range(1, len(list_data)+1):
            clients_data["client_%s" % i] = list_data[i-1]
            clients_label["client_%s" % i] = list_label[i-1]
            
    elif setting == "NONIID":
        # Sort the train data based on digits label
        sorted_index = np.argsort(y_train)
        y_train = y_train[sorted_index]
        x_train = x_train[sorted_index]
        
        # Split training data into to distribute to 100 clients
        data_shards = np.array_split(x_train, 100)
        label_shards = np.array_split(y_train, 100)
        
        # Distribute shards to all clients
        N_CLIENTS = 100
        clients_data = {}
        clients_label = {}
        for i in range(1, N_CLIENTS+1):
            clients_data["client_%s" % i] = data_shards[i-1]
            clients_label["client_%s" % i] = label_shards[i-1]
            
    return clients_data, clients_label, x_train, y_train, x_val, y_val, x_test, y_test

# Initiate global model
def initiate_model(model_fam, num_class):
    if model_fam == "2NN":
        nn = NN2Layers()
        active_nn = nn.initiate(num_class)
    elif model_fam == "CNN":
        nn = CNN()
        active_nn = nn.initiate(num_class)
        
    return active_nn

# Scaling weights to the dataset proportion
def scale_weights(num_local_samples, weights, chosen_clients):
    num_total_samples = len(chosen_clients) * num_local_samples
    scaling_factor = num_local_samples / num_total_samples

    # Loop through each layer weight & biases
    scaled_weights = []
    for component in weights:
        scaled_weights.append(scaling_factor * component)

    return scaled_weights

# Sum all the scaled weights from all clients
def sum_scaled_weights(scaled_weights):
    final_weights = []
    for component in zip(*scaled_weights):
        final_weights.append(tf.math.reduce_sum(component, axis=0))
  
    return final_weights

# Custom global model evaluation
def evaluate_model(model, test_data, test_label):
    
    # Predict label
    y_pred = model.predict(test_data)
    y_true = test_label

    # Calculate loss with SCCE
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    loss = scce(y_true, y_pred).numpy()

    # Calculate accuracy
    accuracy = accuracy_score(y_true, np.argmax(y_pred, axis=1))

    return np.round(loss, 4), np.round(accuracy, 4)

In [4]:
def run_experiment(data="MNIST", use_val=False, model_family="2NN", setting="IID", b_size=600, epoch=1,\
                   est_comm_round=1000, c_fraction=0.1, desired_acc=False,\
                   model_checkpoint=False, n_clients=100, n_classes=10,\
                   learning_rate=0.01, output_path='./logs/', verbose=0):
    
    # Training setting
    EXP_NAME = "%s-B%s-E%s-C%s-MNIST-%s-LR%s" % (setting, b_size, epoch, c_fraction,\
                                                 model_family, learning_rate)
    start_time = time()
    history = []
    comm_round = 1
    
    # Generate data
    clients_data, clients_label, x_train, y_train, x_val, y_val, x_test, y_test = generate_data(data, use_val, setting,\
                                                                                  n_clients)
    
    # Initiate new global model
    if model_checkpoint == False: 
        global_nn = initiate_model(model_family, n_classes)
        
    # Continue training from existing checkpoint
    else:
        global_nn = tf.keras.models.load_model(model_checkpoint)
        
    # Training setting
    comm_round = 1
    start_time = time()
    
    # Loop until estimated communication round reached
    for i in tqdm(range(est_comm_round)):

        # Save the global weight
        global_weights = global_nn.get_weights()

        # Client set
        if c_fraction == 0.0: # Exactly one client
            chosen_clients = np.random.choice(list(clients_data.keys()), 1)
        elif c_fraction == 1.0: # All clients
            chosen_clients = clients_data.keys()
        else: # Percentage subset
            chosen_clients = np.random.choice(list(clients_data.keys()), \
                                          int(c_fraction*len(clients_data)), replace=False)

        clients_weight = []
        for client in chosen_clients:
            # Iniatiate local model
            local_nn = initiate_model(model_family, n_classes)
            optimizer = tf.keras.optimizers.SGD(
                learning_rate=learning_rate, clipnorm=1.0)
            local_nn.compile(
                optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )

            # Set global weight to the local model
            local_nn.set_weights(global_weights)

            # Do training on local
            local_nn.fit(clients_data[client], clients_label[client], batch_size=b_size,\
                         epochs=epoch, verbose=verbose)

            # Save weight
            scaled_weights = scale_weights(len(clients_data[client]), local_nn.get_weights(),\
                                           chosen_clients)
            clients_weight.append(scaled_weights)

        # Sum all scaled weights & update the global model
        global_nn.set_weights(sum_scaled_weights(clients_weight))

        # Evaluate the loss & accuracy
        train_loss, train_accuracy = evaluate_model(global_nn, x_train, y_train)
        test_loss, test_accuracy = evaluate_model(global_nn, x_test, y_test)
        
        # If gradient explode, rollback weight, continue to next round
        if comm_round > 1: 
            if str(test_loss) == "nan":
                print("Explode,", test_accuracy, "<-", history[-1]['test_acc'])
                break

        # Save metrics for current round
        data = {"C": c_fraction, "B": b_size, "learning_rate": learning_rate, 
                "comm_round": comm_round, "train_acc": train_accuracy, 
                "test_acc": test_accuracy, "train_loss": train_loss, "test_loss": test_loss}
        
        # If validation set exists
        if x_val is not None:
            val_loss, val_accuracy = evaluate_model(global_nn, x_val, y_val)
            data['val_acc'] = val_accuracy
            data['val_loss'] = val_loss
        
        # Save model state and history
        history.append(data)
        global_nn.save(output_path + EXP_NAME)
        file = open(output_path + EXP_NAME + ".txt", 'a')
        file.write(json.dumps(str(data)) + "\n")
        file.close()
        print(data)

        # Stop when the desired test-accuracy reached, comm_round ignored
        if desired_acc != False and test_accuracy >= desired_acc:
            break

        # Update variables
        comm_round += 1
    end_time = time()
    print("Took %.2f seconds." % (end_time - start_time))
    return history

In [ ]:
# Run experiment with your own setting
history = run_experiment(data="MNIST", use_val=False, model_family="CNN",\
                         setting="IID", b_size=600, epoch=5,\
                         est_comm_round=300, c_fraction=0.1, desired_acc=0.99,\
                         model_checkpoint="logs/IID-B600-E5-C0.1-MNIST-CNN-LR0.215",\
                         n_clients=100, n_classes=10,\
                         learning_rate=0.215, output_path='./logs/', verbose=0)

  0%|          | 0/300 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: ./logs/IID-B600-E5-C0.1-MNIST-CNN-LR0.215/assets


  0%|          | 1/300 [00:30<2:29:38, 30.03s/it]

{'C': 0.1, 'B': 600, 'learning_rate': 0.215, 'comm_round': 1, 'train_acc': 0.9786, 'test_acc': 0.9776, 'train_loss': 0.0698, 'test_loss': 0.0676}
INFO:tensorflow:Assets written to: ./logs/IID-B600-E5-C0.1-MNIST-CNN-LR0.215/assets


  1%|          | 2/300 [00:59<2:27:53, 29.78s/it]

...

{'C': 0.1, 'B': 600, 'learning_rate': 0.215, 'comm_round': 196, 'train_acc': 0.9946, 'test_acc': 0.9878, 'train_loss': 0.0172, 'test_loss': 0.035}
INFO:tensorflow:Assets written to: ./logs/IID-B600-E5-C0.1-MNIST-CNN-LR0.215/assets


 66%|██████▌   | 197/300 [1:47:02<1:01:12, 35.65s/it]

{'C': 0.1, 'B': 600, 'learning_rate': 0.215, 'comm_round': 197, 'train_acc': 0.9947, 'test_acc': 0.9883, 'train_loss': 0.0172, 'test_loss': 0.0345}
INFO:tensorflow:Assets written to: ./logs/IID-B600-E5-C0.1-MNIST-CNN-LR0.215/assets


 66%|██████▌   | 198/300 [1:47:39<1:01:26, 36.14s/it]

{'C': 0.1, 'B': 600, 'learning_rate': 0.215, 'comm_round': 198, 'train_acc': 0.9948, 'test_acc': 0.9885, 'train_loss': 0.0171, 'test_loss': 0.0343}
